## Đọc dữ liệu

In [12]:
import pandas as pd

df = pd.read_csv("test.csv")
texts = df["text"]
df.head()

,text,summary
0,Sau bài thi ngắn năm mươi phút và bốn mươi câu...,"Đại thụ của nền sử học ra đi, để lại cho cuộc ..."
1,"Với doanh thu bình quân 200 USD/TEU, theo số l...","Với doanh thu bình quân 200 USD/TEU, theo số l..."
2,Tôi có một shop hoa. Ngày Phụ nữ Việt Nam luôn...,Chỉ là bởi giữa lúc ngập tràn trên mạng xã hội...
3,Nhưng ông nông dân miệt vườn này vẫn điềm tĩnh...,"Các nhà khoa học nhận diện, hiện tượng ""ngập t..."
4,"George Black lặng lẽ khóc, khi nghe bác sĩ Bản...",Ông rất tâm huyết với công cuộc hòa giải và hà...


## Hàm tách câu thủ công

In [13]:
import numpy as np
import re

def split_sentences(text):
    sentences = re.split(r"[.!?]\s+", text)
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

## Hàm tính vector Bag-of-Words thủ công

In [14]:
def build_vectors(sentences):
    vocab = {}
    for s in sentences:
        for w in s.lower().split():
            vocab[w] = vocab.get(w, 0) + 1

    vocab_list = list(vocab.keys())
    index = {w:i for i,w in enumerate(vocab_list)}

    def vec(sentence):
        v = np.zeros(len(vocab_list))
        for w in sentence.lower().split():
            if w in index:
                v[index[w]] += 1
        return v

    vectors = [vec(s) for s in sentences]
    return np.array(vectors)

## Cosine similarity

In [15]:
def cosine_sim(a, b):
    if np.linalg.norm(a)==0 or np.linalg.norm(b)==0:
        return 0
    return np.dot(a,b) / (np.linalg.norm(a)*np.linalg.norm(b))

## PageRank

In [16]:
def pagerank(M, eps=0.0001, d=0.85):
    N = M.shape[0]
    score = np.ones(N) / N
    while True:
        new_score = (1 - d)/N + d * M.T.dot(score)
        if np.linalg.norm(new_score - score) < eps:
            break
        score = new_score
    return score

## Thuật toán TextRank

In [17]:
def textrank_summary(text, k=3):
    sentences = split_sentences(text)
    if len(sentences) <= k:
        return text  # văn bản quá ngắn

    vectors = build_vectors(sentences)

    n = len(sentences)
    sim = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i != j:
                sim[i][j] = cosine_sim(vectors[i], vectors[j])

    # Ma trận chuyển tiếp
    row_sum = sim.sum(axis=1)
    M = np.zeros((n,n))
    for i in range(n):
        if row_sum[i] != 0:
            M[i] = sim[i] / row_sum[i]

    # PageRank
    scores = pagerank(M)

    # Lấy k câu quan trọng nhất
    idx = scores.argsort()[-k:][::-1]
    summary = ". ".join([sentences[i] for i in idx])
    return summary

## Chạy thuật toán TextRank

In [18]:
summaries = []
for i, text in enumerate(texts):
    print(f"Đã xử lý văn bản {i+1}/{len(texts)} ...")
    s = textrank_summary(text, k=3)
    summaries.append(s)

df["textrank_summary"] = summaries

Đã xử lý văn bản 1/178 ...
Đã xử lý văn bản 2/178 ...
Đã xử lý văn bản 3/178 ...
Đã xử lý văn bản 4/178 ...
Đã xử lý văn bản 5/178 ...
Đã xử lý văn bản 6/178 ...
Đã xử lý văn bản 7/178 ...
Đã xử lý văn bản 8/178 ...
Đã xử lý văn bản 9/178 ...
Đã xử lý văn bản 10/178 ...
Đã xử lý văn bản 11/178 ...
Đã xử lý văn bản 12/178 ...
Đã xử lý văn bản 13/178 ...
Đã xử lý văn bản 14/178 ...
Đã xử lý văn bản 15/178 ...
Đã xử lý văn bản 16/178 ...
Đã xử lý văn bản 17/178 ...
Đã xử lý văn bản 18/178 ...
Đã xử lý văn bản 19/178 ...
Đã xử lý văn bản 20/178 ...
Đã xử lý văn bản 21/178 ...
Đã xử lý văn bản 22/178 ...
Đã xử lý văn bản 23/178 ...
Đã xử lý văn bản 24/178 ...
Đã xử lý văn bản 25/178 ...
Đã xử lý văn bản 26/178 ...
Đã xử lý văn bản 27/178 ...
Đã xử lý văn bản 28/178 ...
Đã xử lý văn bản 29/178 ...
Đã xử lý văn bản 30/178 ...
Đã xử lý văn bản 31/178 ...
Đã xử lý văn bản 32/178 ...
Đã xử lý văn bản 33/178 ...
Đã xử lý văn bản 34/178 ...
Đã xử lý văn bản 35/178 ...
Đã xử lý văn bản 36/178 ...
Đ

In [19]:
for i in range(len(df)):
    print("="*80)
    print(f" VĂN BẢN GỐC {i+1}:")
    print(df['text'].iloc[i])
    print("\n TÓM TẮT TEXT RANK:")
    print(df['textrank_summary'].iloc[i])
    print("="*80, "\n")

 VĂN BẢN GỐC 1:
Sau bài thi ngắn năm mươi phút và bốn mươi câu hỏi trắc nghiệm, phần lớn họ, nếu không chọn ngành khoa học xã hội ở đại học, sẽ tạm dừng “mối duyên” với lịch sử trong nhiều năm về sau này.
Khi hàng trăm nghìn cái đầu trẻ chăm chú cúi xuống một bài thi lịch sử, tôi nhớ tới giáo sư Phan Huy Lê, người rời cõi tạm ngày 23/6. Đại thụ của nền sử học ra đi, để lại cho cuộc đời một câu hỏi: bao giờ những khoảng trống lịch sử được lấp đầy?
Trong bốn mươi câu trắc nghiệm lịch sử sáng nay, sẽ còn thiếu rất nhiều sự kiện và sự thật lịch sử quan trọng của Việt Nam, mà ngành giáo dục vẫn còn chưa thống nhất được với ngành lịch sử, với những nhà nghiên cứu trăn trở như Phan Huy Lê, rằng chúng có nên được đưa vào và sẽ được đưa vào sách giáo khoa như thế nào.
Lần đầu tiên tôi nhận ra sự thiếu hụt trong kiến thức lịch sử của mình, là khi đã trở thành sinh viên năm thứ ba của trường báo. Đó là lần đầu tiên tôi được đọc cuốn sách về phong trào Nhân văn giai phẩm. Đó là lần đầu tiên tôi bi

# Đánh giá độ hiệu quả của thuật toán

In [20]:
!pip install rouge-score bert-score scikit-learn

In [21]:
from rouge_score import rouge_scorer
from bert_score import score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

gold = df["summary"].astype(str).tolist()

#TextRank summary
pred = df["textrank_summary"].astype(str).tolist()

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge1_list = []
rouge2_list = []
rougeL_list = []

for g, p in zip(gold, pred):
    scores = scorer.score(g, p)
    rouge1_list.append(scores["rouge1"].fmeasure)
    rouge2_list.append(scores["rouge2"].fmeasure)
    rougeL_list.append(scores["rougeL"].fmeasure)

avg_rouge1 = sum(rouge1_list) / len(rouge1_list)
avg_rouge2 = sum(rouge2_list) / len(rouge2_list)
avg_rougeL = sum(rougeL_list) / len(rougeL_list)

# 3. BERTScore

P, R, F1 = score(pred, gold, lang="vi")   # lang="vi" cho tiếng Việt

bert_f1 = float(F1.mean())

# Cosine similarity

vectorizer = TfidfVectorizer()
cosine_scores = []

for g, p in zip(gold, pred):
    tfidf = vectorizer.fit_transform([g, p])
    cosine = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
    cosine_scores.append(cosine)

avg_cosine = sum(cosine_scores) / len(cosine_scores)

# Đánh giá
print("=== KẾT QUẢ ĐÁNH GIÁ TEXT RANK ===\n")
print(f"ROUGE-1 (F1): {avg_rouge1:.4f}")
print(f"ROUGE-2 (F1): {avg_rouge2:.4f}")
print(f"ROUGE-L (F1): {avg_rougeL:.4f}")
print(f"BERTScore (F1): {bert_f1:.4f}")
print(f"Cosine Similarity: {avg_cosine:.4f}")

=== KẾT QUẢ ĐÁNH GIÁ TEXT RANK ===

ROUGE-1 (F1): 0.5717
ROUGE-2 (F1): 0.5390
ROUGE-L (F1): 0.4528
BERTScore (F1): 0.7953
Cosine Similarity: 0.7394


## Ví dụ minh họa

In [22]:
examples = [
"""Ngày hôm nay, Ngân hàng Nhà nước bất ngờ điều chỉnh tăng tỷ giá trung tâm thêm 50 đồng.
Động thái này diễn ra trong bối cảnh đồng USD trên thị trường quốc tế tiếp tục mạnh lên.
Nhiều chuyên gia cho rằng việc điều chỉnh tỷ giá là cần thiết để giữ ổn định nền kinh tế
và bảo vệ xuất khẩu của Việt Nam. Dù vậy, việc tỷ giá tăng có thể gây áp lực lên lạm phát
và giá hàng nhập khẩu trong thời gian tới.""",

"""Bộ Giáo dục và Đào tạo cho biết kỳ thi tốt nghiệp THPT năm nay sẽ không thay đổi cấu trúc đề,
nhằm tạo sự ổn định cho học sinh. Tuy nhiên, nội dung đề thi sẽ tăng cường các câu hỏi vận dụng
và vận dụng cao, yêu cầu thí sinh phải hiểu bản chất kiến thức thay vì học thuộc lòng.
Nhiều giáo viên nhận định điều này giúp đánh giá đúng năng lực học sinh hơn."""
]

for i, text in enumerate(examples):
    print(f"*VÍ DỤ {i+1}")
    print("\n")
    print("VĂN BẢN GỐC:")
    print(text)

    summary = textrank_summary(text, k = 2)

    print("\n TÓM TẮT TEXT RANK:")
    print(summary)

*VÍ DỤ 1


VĂN BẢN GỐC:
Ngày hôm nay, Ngân hàng Nhà nước bất ngờ điều chỉnh tăng tỷ giá trung tâm thêm 50 đồng.
Động thái này diễn ra trong bối cảnh đồng USD trên thị trường quốc tế tiếp tục mạnh lên.
Nhiều chuyên gia cho rằng việc điều chỉnh tỷ giá là cần thiết để giữ ổn định nền kinh tế
và bảo vệ xuất khẩu của Việt Nam. Dù vậy, việc tỷ giá tăng có thể gây áp lực lên lạm phát
và giá hàng nhập khẩu trong thời gian tới.

 TÓM TẮT TEXT RANK:
Dù vậy, việc tỷ giá tăng có thể gây áp lực lên lạm phát
và giá hàng nhập khẩu trong thời gian tới.. Ngày hôm nay, Ngân hàng Nhà nước bất ngờ điều chỉnh tăng tỷ giá trung tâm thêm 50 đồng
*VÍ DỤ 2


VĂN BẢN GỐC:
Bộ Giáo dục và Đào tạo cho biết kỳ thi tốt nghiệp THPT năm nay sẽ không thay đổi cấu trúc đề,
nhằm tạo sự ổn định cho học sinh. Tuy nhiên, nội dung đề thi sẽ tăng cường các câu hỏi vận dụng 
và vận dụng cao, yêu cầu thí sinh phải hiểu bản chất kiến thức thay vì học thuộc lòng.
Nhiều giáo viên nhận định điều này giúp đánh giá đúng năng lực học 